# This is just importing packages and setting up Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install pytorch-lightning==1.3.3
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl
!pip install scGeneFit
!pip install scanpy
!pip install anndata
!pip install lassonet

In [ ]:
# Check GPU

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
import sys
import os
# to import utils
sys.path.append(os.path.abspath('/content/gdrive/My Drive/Differentiable-Sparse-Subset-Selection/notebooks'))

In [ ]:
%%time 
import itertools 
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F

from utils import *
from sklearn.preprocessing import LabelEncoder


# Defining some parameters.

In [ ]:
z_size = 16
hidden_layer_size = 256
k = 50
gpus = 1


# Load the data

Didn't need to pre-process since Zeisel is well processed, but the `process_data` function does exist!

In [ ]:
dataset_dir = '/content/gdrive/MyDrive/Differentiable-Sparse-Subset-Selection/data/zeisel/'

In [ ]:
adata = sc.read_h5ad(dataset_dir + "Zeisel.h5ad")

In [ ]:
X = adata.X.copy()

labels = adata.obs['names0'].values
encoder = LabelEncoder()
encoder.fit(labels)
Y = encoder.transform(labels)

In [ ]:
input_size = X.shape[1]
num_classes = len(encoder.classes_)

# Train MarkerMap in 3 Settings 

### Just a demo. Don't take the metrics seriously because I didn't use a test set.

## Purely unsupervised

In [ ]:
marker_map = MarkerMap(input_size = input_size, 
                       hidden_layer_size = hidden_layer_size, 
                       z_size = z_size, 
                       k = k,          
                       # PURELY UNSUPERVISED
                       num_classes = None,
                       loss_tradeoff = 1.0)


### Train using all the data. No early stopping with validation set

In [ ]:
markers, classifier, extras = marker_map.map(X = X, Y = Y, train_ratio = 1.0, gpus = gpus, min_epochs = 10, max_epochs = 25)

In [ ]:
markers

In [ ]:
np.sum(classifier.predict(X[:, markers]) == Y) / len(Y)

### Not sure what should go in extras

In [ ]:
print(extras)

###

## Purely supervised

In [ ]:
marker_map = MarkerMap(input_size = input_size, 
                       hidden_layer_size = hidden_layer_size, 
                       z_size = z_size, 
                       k = k,          
                       # PURELY Supervised
                       num_classes = num_classes,
                       loss_tradeoff = 0)


### This time, 80% of the training data is used as validation data for early stopping of the model

In [ ]:
markers, classifier, extras = marker_map.map(X = X, Y = Y, train_ratio = 0.8, gpus = gpus, min_epochs = 10, max_epochs = 25)

### Use the model itself to predict classes. 

In [ ]:
np.sum(marker_map.predict_class(X) == Y) / len(Y)

### Can also use the classifier

In [ ]:
classifier.predict(X[:, markers])

## Mix of supervised and unsupervised

In [ ]:
from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [ ]:
marker_map = MarkerMap(input_size = input_size, 
                       hidden_layer_size = hidden_layer_size, 
                       z_size = z_size, 
                       k = k,          
                       # PURELY Supervised
                       num_classes = num_classes,
                       loss_tradeoff = 0.5)


In [ ]:
markers, classifier, extras = marker_map.map(X = X, Y = Y, train_ratio = 0.8, gpus = gpus, min_epochs = 10, max_epochs = 25)

# Do a plot from the Mix Supervised/Unsupervised setting

In [ ]:
if markers is not None:
    X_viz = X[:, markers]
embedding = umap.UMAP(n_neighbors=10, min_dist= 0.05).fit_transform(X_viz)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8.5))

embedding_X, embedding_Y = embedding.T
# plt.setp(ax, xticks=[], yticks=[])
for class_i in range(num_classes):
    label = encoder.classes_[class_i]
    indices = Y == class_i
    plt.scatter(embedding_X[indices], embedding_Y[indices], label = label)
    
#cbar = plt.colorbar(ticks=np.arange(num_classes))#, boundaries = np.arange(num_classes) - 0.5)
#cbar.ax.set_yticklabels(encoder.classes_)


plt.legend()
plt.title('UMAP of 50 Markers from MarkerMap')
plt.show()